In [13]:
import pandas as pd
import spacy 
nlp = spacy.load("en_core_web_sm")

AttributeError: type object 'Language' has no attribute 'factory'

In [ ]:
df  = pd.read_csv('books_with_blurbs.csv')

In [ ]:
df.head()

In [ ]:
df['Blurb'][10:15]

In [ ]:
# function to clean data

def cleanData(doc,stemming = False):
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [token.lemma_ for token in tokens]
    
    return " ".join(final_token)

In [ ]:
print("Cleaning train data...\n")
df["blurb_clean"] = df["Blurb"].map(lambda x: cleanData(x))

In [ ]:
df.head()

In [ ]:
df['Title_lower'] = df['Title'].apply(lambda x: x.lower())

In [ ]:
df['blurb_clean_title'] =  df['Title_lower'] +  ' ' + df['blurb_clean']

In [ ]:
df['blurb_clean_title'][10]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['blurb_clean_title'])

In [ ]:
df1 = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
print(df1)

In [ ]:
def calculate_cosine_sim(sentence):
    vectorizer2 = TfidfVectorizer()
    sentence = input("Write a line from your favorite book?")
    Y = vectorizer2.fit_transform(sentence)
    